In [1]:
# Zum installieren von beatifulsoup benötigt
#! pip install beautifulsoup4



In [2]:
#Für das Projekt verwendete Webseite 
#Dieser String wird später zum aufruf sämtlicher weiteren URL's verwendet
page = 'https://www.stepstone.de'

In [3]:
#Alle relevanten Sprachen und Kompetenzen nach denen in den Jobanzeigen gesucht wird
defaultLangueList =   ['Java', 'C#' , 'C++' , 'HTML','Python','C' ,
               'PHP', 'JavaScript', 'SQL', 'CSS']

#Eine Seite, in denen die Jobanzeigen augelistet werden, hat bis zu 100 Links zu den Jobanzeigen 
#numberOfSearch steht für die Anzahl der JobListenSeiten die Abgearbeitet werden 
numberOfSearch = 20



#die Absolute Zahl aller durchsuchten Jobanzeigen
totalNumberOfAnnonces = 0 
#die Absolute Zahl aller durchsuchten Jobanzeigen bei denen mindestens eine Sprache gefunden wurde
conformAnnoces = 0

#Um groß und Kleinschreibung zu ignorieren verwende ich eine liste in denen alle namen klein geschrieben sind
langueList =  []
for i in range (0,len(defaultLangueList)):
    langueList.append(defaultLangueList[i].lower())

#createPointList erstellt für jede Sprache in der SprachenListe hier drüber einen Integer 0 in der Liste pointList
#diese Werte werden später für das absolute und relativeErgebnis der jeweiligen Sprache verwendet 
def createPointList():
    pointList =[]
    for item in langueList:
        pointList.append(0)
    return pointList    
pointList = createPointList()    

#createFoundList wird für eine Booleanliste verwendet die für jede Sprache einen Booleanwert hat
#falls eine Sprache gefunden wird wird der entsprechende Platz in der Foundlist auf true gesetzt 
#bei jeder neuen anzeige wird eine neue foundlist erstellt
def createFoundList():
    foundList =[]
    for item in langueList:
        foundList.append(False)
    return foundList    


   
#mergeList wird für die Ausgabe verwendet und schreibt den absoluten oder
#relativen wert zusammen mit der dazugehörigen sprache in die ausgabe
def mergeList(langue, point):
    if (len(langue) != len(point) ):
        print("Fehler: beide Listen sollten Gleichlang sein")
    for i in range (0,len(langue)):
        print (str(langue[i])+ ": " + str(point[i]))
        
        

In [4]:
from bs4 import BeautifulSoup
from datetime import datetime
import sys
import re
import requests
import lxml.html as lh

#funktion getReport um eine aktuelle Auswertung der Jobanzeigen vorzunehmen 
#variable url ist die erste Seite für die Anzeige für Softwareentwickler 
#von hier aus beginnt die funktion die suche nach allen url's der Jobanzeigen
def getReport(url):
    global page ,pointList,defaultLangueList
    pointList = createPointList()
    urlList = getUrlFromWebsite(url,0)
    #print(urlList)
    print("Starte Anzeigen-Analyse")
    for link in urlList:
        text = getItemsFromURL(page+link)
       
        analyseText(text)
    doComprehension()
    
#Extrahiert die wichtigen Listenobbjekte der Seite und gibt den inhalt dieser zurück
#url ist hier die url der Jobanzeige 
def getItemsFromURL(url):
        try:
            page = requests.post(url)
        
            soup = BeautifulSoup(page.content, 'html.parser')
            week = []
            week = soup.find(class_ = 'js-app-ld-ContentBlock')
            if not str(week) :
                print ("gesuchte Klasse existiert nicht und muss ausgetauscht werden")
            text = ''
            for tag in week.find_all("li",recursive = True):

                s = ' '+ str(tag.text)+ ' '
                text = text + re.sub('[\d,.)(/-]', ' ', s)
        except: 
            return ''
        else:
            return text
#Tritt auf wenn der zu durchsuchenden Seiten erreicht wurde    
class NoNextSite(Exception):
    """Tritt auf wenn die Seitensuche ihr maximum erreicht hat"""
    pass
    


#getUrlFromWebsite sucht alle URL's für Jobanzeigen auf der übergebenen seite url
#und ruft sich rekursiv selber auf bis die maximalanzahl von Seitensuchen numberSearched erreicht wurde
def getUrlFromWebsite(url, numberSearched):
    global numberOfSearch
    page = requests.post(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    urlList = []
    for a in soup.find_all('a', href=True):
        if ('stellenangebote' in str(a) and a['href'] not in urlList):
            urlList.append (a['href'])
          
    newUrl = soup.find(direction = 'right')
   
    try: 
        testNextUrL = newUrl['href']
        if numberOfSearch-1 == numberSearched:
            raise NoNextSite
    except KeyError:
        print("URL-Suche beendet")
    except NoNextSite:
        print("URL-Suche beendet")
    else:
        
        urlList = urlList + getUrlFromWebsite(newUrl['href'],numberSearched +1)
    
       
   
    return urlList    
   
    
# funktion analysiert ob eine sprache in einem text genannt wird 
# in diesem fall wird davon ausgegangen dass genannt = vorraussetzung für den Beruf ist
def analyseText(text):
    global langueList , pointList, totalNumberOfAnnonces,conformAnnoces
    foundList = createFoundList()
    for word in str(text).split(' ') :
        if(word.lower() in langueList):
            for i in range (len(langueList)):
                if (langueList[i] == word.lower() and foundList[i] == False):
                    pointList[i] = pointList[i] +1 
                    foundList[i] = True         
    if(True in foundList):
        conformAnnoces = conformAnnoces +1
    totalNumberOfAnnonces = totalNumberOfAnnonces +1    
    
    
#doComprehension fasst die Ergebnisse zusammen und gibt diese dann mit mergeList aus    
def doComprehension():
    global pointList, totalNumberOfAnnonces,conformAnnoces
    mergeList(defaultLangueList, pointList) 
    print("Bei " + str(totalNumberOfAnnonces) +" überprüften Anzeigen, wovon bei " + str(conformAnnoces) + 
          " mindestens eine dem Programm bekannte Sprachen dabei waren.")
    for i in range (len(pointList)):
        pointList[i] = str(100*pointList[i]/conformAnnoces)[:5]+ "%" 
    mergeList(defaultLangueList, pointList) 




In [5]:
#url mit der Anzeigenmenge 25 pro Seite
#url = 'https://www.stepstone.de/5/ergebnisliste.html?stf=freeText&ns=1&qs=%5B%7B%22id%22%3A%22216805%22%2C%22description%22%3A%22Software-Entwickler%2Fin%22%2C%22type%22%3A%22jd%22%7D%5D&companyID=0&cityID=0&sourceOfTheSearchField=resultlistpage%3Ageneral&searchOrigin=Resultlist_top-search&ke=Software-Entwickler%2Fin&ws=&ra=30'
#url mit der Anzeigenmenge 100 pro Seite
url = 'https://www.stepstone.de/5/ergebnisliste.html?stf=freeText&ns=1&companyid=0&sourceofthesearchfield=resultlistpage%3Ageneral&qs=[{"id"%3A216805%2C"description"%3A"Software-Entwickler\%2Fin"%2C"type"%3A"jd"}]&ke=Software-Entwickler%2Fin&ra=30&suid=c4f83374-3466-4871-be7a-c171dfa6cb57&li=100&of=0&action=per_page_changed'
getReport(url)

URL-Suche beendet
starte Anzeigen Analyse
Java: 688
C#: 398
C++: 466
HTML: 287
Python: 231
C: 383
PHP: 103
JavaScript: 382
SQL: 422
CSS: 215
Bei 2000 überprüften Anzeigen, wovon bei 1565 mindestens eine dem Programm bekannte Sprachen dabei waren.
Java: 43.96%
C#: 25.43%
C++: 29.77%
HTML: 18.33%
Python: 14.76%
C: 24.47%
PHP: 6.581%
JavaScript: 24.40%
SQL: 26.96%
CSS: 13.73%
